In [ ]:
!pip install torch torchvision

In [ ]:
import time
import torch
import torchvision
from torchvision import transforms, datasets
import matplotlib.pyplot as plt
from torch import nn

In [ ]:
# Used https://d2l.ai/chapter_linear-classification/image-classification-dataset.html
# Used https://www.kaggle.com/code/leifuer/intro-to-pytorch-fashion-mnist

In [ ]:
#Define our data
#transform converts image to 28 by 28 and turns them to a tensor
data_transform = transforms.Compose([transforms.Resize((28, 28)),
                                 transforms.ToTensor()])


training_data = datasets.FashionMNIST('./datasets/fashionMNIST', download=True, train=True, transform=data_transform)

training_dataloader = torch.utils.data.DataLoader(training_data, 64, shuffle=True)

test_data = datasets.FashionMNIST('./datasets/fashionMNIST', download=True, train=False, transform=data_transform)

test_dataloader = torch.utils.data.DataLoader(training_data, 64, shuffle=True)



In [ ]:
# image helpers(from d2l)
def text_labels(indices):
    """Return text labels."""
    labels = ['t-shirt', 'trouser', 'pullover', 'dress', 'coat',
              'sandal', 'shirt', 'sneaker', 'bag', 'ankle boot']
    return [labels[int(i)] for i in indices]

def show_images(imgs, num_rows, num_cols, titles=None, scale=1.5):  #@save
    """Plot a list of images."""
    figsize = (num_cols * scale, num_rows * scale)
    _, axes = plt.subplots(num_rows, num_cols, figsize=figsize)
    axes = axes.flatten()
    for i, (ax, img) in enumerate(zip(axes, imgs)):
        try:
            img = img.detach().numpy()
        except:
            pass
        ax.imshow(img)
        ax.axes.get_xaxis().set_visible(False)
        ax.axes.get_yaxis().set_visible(False)
        if titles:
            ax.set_title(titles[i])
    return axes
def visualize(batch, nrows=1, ncols=8, labels=[]):
    X, y = batch
    if not labels:
        labels = text_labels(y)
    show_images(X.squeeze(1), nrows, ncols, titles=labels)

In [ ]:
visualize(next(iter(training_dataloader)))

In [ ]:
#define our softmax
class Softmax(nn.Module):
    def __init__(self, number_of_outputs, learning_rate):
        super().__init__()
        self.net = nn.Sequential(nn.Flatten(),
                                nn.LazyLinear(number_of_outputs))
        self.learning_rate = learning_rate
    
    def forward(self, x):
        return self.net(x)


In [ ]:
model = Softmax(10, 1e-5)

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), model.learning_rate)

In [ ]:
loss_function = nn.CrossEntropyLoss()
    

In [ ]:
#now we can define a train

def train(epoch, training_loader):
    for i in range(epoch):
        for x, y in training_loader:
            prediction = model(x)
            loss = loss_function(prediction, y)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            print(i)
            print(loss)
        

In [ ]:
#Lets train our model
optimizer.zero_grad()
train(100, training_dataloader)


In [ ]:
# Now let's see how we did!
total = 0
correct = 0
with torch.no_grad():
    model.eval()
    for images, labels in test_dataloader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))
